Université Paul Sabatier

EIMAB3H1 - Analyse et exploitation de données

Enseignant : **José G. Moreno**

## TP 1. Extraction d'information sur documents textuels (1/3)
Identifier correctement la présence de noms ou informations dans un texte est une tâche important de
l'extraction d'information. Par exemple dans le texte

```
« Le Front Fatah Al-Cham, qui prétend avoir rompu avec Al-Qaida, mais que 
Washington considère toujours comme terroriste, occupe une place centrale sur
l’échiquier rebelle. D’où la difficulté à départager bons et mauvais rebelles. En
savoir plus sur http://www.lemonde.fr/#ltXX2jh9UwX7vbDM.99 »
```
, les noms ou informations à identifier sont :
- Front Fatah Al-Cham -> Organisation
- Al-Qaida -> Organisation
- Washington -> Lieu
- http://www.lemonde.fr/#ltXX2jh9UwX7vbDM.99 -> URL

ou dans le texte
```
« Un futur pont sur la Saône sera baptisé du nom de Jacques Chirac « en hommage
à l’attachement de l’ancien président de la République aux territoires et à leur unité
», ont annoncé mardi les départements de l’Ain et de la Saône-et-Loire.
Disparu jeudi, « Jacques Chirac était de ceux qui savaient écouter, comprendre et
agir pour le bien de ses concitoyens », soulignent Jean Deguerry, président du
département de l’Ain, et André Accary, président du département de Saône-et-Loire,
dans un communiqué commun ».
```

- Saône -> Lieu
- Jacques Chirac -> Personne
- Ain -> Lieu
- Saône-et-Loire -> Lieu
- Jean Deguerry -> Personne
- André Accary -> Personne

Dans ces deux exemples, nous nous limitons à quatre types de noms ou informations (Organisation,
Lieu, Personne et URL), cependant des autres informations peuvent nous intéresser comme les dates,
les adresses mails, les noms de produits, les événements, etc, dans la littérature, ces noms ou
informations sont appelés des « entités nommées ». 

### I - L'usage des dictionnaires
Pour identifier certains types d’entités nommées l'utilisation de dictionnaires est courante. Par exemple,
l'identification d'un lieu comme un ville ou un pays. Nous allons construire un dictionnaire de pays à
partir de Wikipédia. Wikipédia est une des ressources disponibles sur Internet le plus complètes et de
libre accès. Elle est aussi très utilisé comme source d'information par plusieurs entreprises. Par exemple
le projet Freebase de Google a été créé à partir de donnés extrait de Wikipédia parmi autres sources.
Aujourd'hui devenu le Knowledge Graph est la base de connaissance utilisé par Google pour améliorer
son moteur de recherche.
Parmi toutes les informations disponibles sur Wikipédia, la listes de listes est une source important
d'information (pour la Wikipédia en anglais utilisez l'article List of list of list). Avec l'article respective,
nous allons générer un dictionnaire de la liste des pays par exemple :


In [6]:
!wget https://dumps.wikimedia.org/simplewiki/20200901/simplewiki-20200901-pages-articles.xml.bz2
!bzip2 -d simplewiki-20200901-pages-articles.xml.bz2
! echo "<mediawiki>" > simplewiki-20200901-pages-articles.xml.clean
! tail -n +2 simplewiki-20200901-pages-articles.xml >> simplewiki-20200901-pages-articles.xml.clean

--2020-11-08 13:38:11--  https://dumps.wikimedia.org/simplewiki/20200901/simplewiki-20200901-pages-articles.xml.bz2
Resolving dumps.wikimedia.org (dumps.wikimedia.org)... 208.80.154.7, 2620:0:861:1:208:80:154:7
Connecting to dumps.wikimedia.org (dumps.wikimedia.org)|208.80.154.7|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 179777498 (171M) [application/octet-stream]
Saving to: ‘simplewiki-20200901-pages-articles.xml.bz2.4’

simplewiki-20200901 100%[===================>] 171.45M  4.86MB/s    in 35s     

2020-11-08 13:38:47 (4.83 MB/s) - ‘simplewiki-20200901-pages-articles.xml.bz2.4’ saved [179777498/179777498]

bzip2: Output file simplewiki-20200901-pages-articles.xml already exists.


In [7]:
import pandas as pd
import xml.etree.ElementTree as et

def parse_XML(xml_file, df_cols): 
    """Parse the input XML file and store the result in a pandas 
    DataFrame with the given columns. 
    
    The first element of df_cols is supposed to be the identifier 
    variable, which is an attribute of each node element in the 
    XML data; other features will be parsed from the text content 
    of each sub-element. 
    """
    
    xtree = et.parse(xml_file)
    xroot = xtree.getroot()
    rows = []
    
    for node in xroot: 
      if node.find(df_cols[0]) is not None:
        res = [node.find(df_cols[0]).text]
        for el in df_cols[1:]: 
            if node is not None and node.find('revision',).find(el,) is not None:
                res.append(node.find('revision',).find(el).text)
            else: 
                res.append(None)
        rows.append({df_cols[i]: res[i] 
                     for i, _ in enumerate(df_cols)})
    
    out_df = pd.DataFrame(rows, columns=df_cols)
        
    return out_df

In [8]:
df = parse_XML("simplewiki-20200901-pages-articles.xml.clean", ["title", "text"])


Nous allons accéder à la page  Wikipédia « List of countries » qui contient la liste des pays :

In [ ]:
print(df[df['title']=='List of countries']['text'].values)

['This is a list of [[sovereign state]]s.\n\n{| class="sortable wikitable" style="background:white; text-align:left;"\n|-\n! style="width:35%" |Common and formal names\n! style="width:12.5%;" |Membership within the [[United Nations System|UN System]]{{efn|This column indicates whether or not a state is a member of the [[United Nations]]. It also indicates which non-member states participate in the [[United Nations System]] through membership in the [[International Atomic Energy Agency]] or one of the [[List of specialized agencies of the United Nations|specialized agencies of the United Nations]]. All United Nations members belong to at least one specialized agency and are parties to the statute of the [[International Court of Justice]].}}\n! style="width:12.5%;" |Sovereignty dispute{{efn|This column indicates whether or not a state is the subject of a major sovereignty dispute. Only states whose entire sovereignty is disputed by another state are listed.}}\n! class="unsortable" |Furth

Voici la liste final dans une variable de type [set](https://docs.python.org/2/library/sets.html)

In [ ]:
import re

s = str(df[df['title']=='List of countries']['text'].values)
pattern = re.compile(r"\[\[([A-Za-z0-9_ ]+)\]\]")

my_countries = set([x.group(0)[2:-2] for x in pattern.finditer(s)])
my_countries

{'Abkhazia',
 'Abyei Area',
 'Aceh',
 'Adjara',
 'African Union',
 'Akrotiri and Dhekelia',
 'Ashmore and Cartier Islands',
 'Australian Antarctic Territory',
 'Autonomous Region in Muslim Mindanao',
 'Azores',
 'BBC News',
 'Bajo Nuevo Bank',
 'Baker Island',
 'Barbuda',
 'Belarus',
 'Bhutan',
 'Bonaire',
 'Bouvet Island',
 'China',
 'China and the United Nations',
 'Chinese Civil War',
 'Chinese Taipei',
 'Clipperton Island',
 'Commonwealth of Nations',
 'Commonwealth realm',
 'Communist Party of China',
 'Compact of Free Association',
 'Comprehensive Peace Agreement',
 'Constitution of Iceland',
 'Coral Sea Islands Territory',
 'Crown dependencies',
 'Cyprus dispute',
 'Dominion',
 'East Jerusalem',
 'Easter Island',
 'Economic Cooperation Organization',
 'Elizabeth II',
 'England',
 'European Union',
 'Federal government of the United States',
 'Federated state',
 'Foreign relations of China',
 'Foreign relations of Cyprus',
 'Foreign relations of North Korea',
 'Foreign relations 

Utilisez le code ci-dessus pour accèder à autres pages et extraire des dictionnaires pour les villes, présidents, personnages fictifs, etc.

In [ ]:
s = str(df[df['title']=='List of cities in France']['text'].values)
my_cities = set([x.group(0) for x in pattern.finditer(s)])
my_cities

{'[[Abbeville]]',
 '[[Agde]]',
 '[[Agen]]',
 '[[Ain]]',
 '[[Aisne]]',
 '[[Ajaccio]]',
 '[[Albi]]',
 '[[Alfortville]]',
 '[[Allier]]',
 '[[Amiens]]',
 '[[Angers]]',
 '[[Anglet]]',
 '[[Annecy]]',
 '[[Annemasse]]',
 '[[Antibes]]',
 '[[Argenteuil]]',
 '[[Arles]]',
 '[[Arras]]',
 '[[Aubagne]]',
 '[[Aube]]',
 '[[Aubervilliers]]',
 '[[Auch]]',
 '[[Aude]]',
 '[[Aurillac]]',
 '[[Auxerre]]',
 '[[Aveyron]]',
 '[[Avignon]]',
 '[[Bagnolet]]',
 '[[Bastia]]',
 '[[Bayonne]]',
 '[[Beaune]]',
 '[[Beauvais]]',
 '[[Belfort]]',
 '[[Bezons]]',
 '[[Biarritz]]',
 '[[Blagnac]]',
 '[[Blois]]',
 '[[Bobigny]]',
 '[[Bondy]]',
 '[[Bordeaux]]',
 '[[Bourges]]',
 '[[Bron]]',
 '[[Brunoy]]',
 '[[Cachan]]',
 '[[Caen]]',
 '[[Calais]]',
 '[[Calvados]]',
 '[[Cambrai]]',
 '[[Cannes]]',
 '[[Cantal]]',
 '[[Carcassonne]]',
 '[[Carpentras]]',
 '[[Castres]]',
 '[[Cavaillon]]',
 '[[Cayenne]]',
 '[[Cenon]]',
 '[[Cergy]]',
 '[[Charente]]',
 '[[Chartres]]',
 '[[Chatou]]',
 '[[Cholet]]',
 '[[Clamart]]',
 '[[Colmar]]',
 '[[Colombes]]',

### II - L'usage des règles
Les listes ou dictionnaires sont utiles pour identifier les entités nommées, mais ce n'est pas le cas dans
la plus part de documents. Dans le deuxième exemple, la chaîne de caractères André Accary est
identifié comme un entité nommée de type personne, mais ce nom n’ai pas partie d'un dictionnaire des
personnes extrait depuis la Wikipédia (car la page n’existe pas). L'usage de règles est une alternative
viable dans ce cas. Par exemple, la règle « mort de TOKEN » (ou sont équivalent en anglais « death
of ») peut être utile pour identifier que TOKEN est une entité nommée de type Personne. TOKEN
peut être aussi défini comme deux mots qui commencent par une majuscule. Voici le code pour
appliquer cette règle sur Wikipédia :

In [ ]:
def findPeople(s): 
  pattern = re.compile(r"death of ([A-Z]\w+( [A-Z]\w+)?)")
  for x in pattern.finditer(str(s)):
    print(str(x.group(0)))

for page in list(df['text']):
  findPeople(page)

death of Qu Yuan
death of Qu Yuan
death of King
death of Mstyslav Volodymyrovych
death of Soviet Union
death of President
death of Prince
death of Hitler
death of Schiller
death of Socrates
death of Louis XII
death of George Washington
death of Crassus
death of Mario Jeckle
death of King
death of Buffy
death of Colonel
death of Moses
death of Marcellus II
death of Stephen
death of Sultan Muhammad
death of King
death of Jesus
death of Freddie Gray
death of Amina
death of Tammi Terrell
death of Glenn Frey
death of Maria
death of Diepndra
death of Mohammed
death of Trujillo
death of Bogd Khaan
death of Candamo
death of Vaikundar
death of Valentine
death of John Cabot
death of John Cabot
death of King
death of Charles XII
death of Milosevic
death of Henry Dashwood
death of Clara Schumann
death of Aurangzeb
death of Patroclus
death of Krishna
death of Charles
death of Jesus
death of Drusus
death of Augustus
death of King
death of Rachel Donelson
death of God
death of Queen
death of Masseria

1) Définir et implémenter des règles pour identifier des entités nommées type Pays et comparer avec la
liste qui nous avons extrait auparavant (sans utiliser la page « List of countries »). Quelle pays ont été
simples à extraire et pourquoi ? Avez-vous réussir à extraire des pays que ne font pas partie de la liste ?
Pourquoi ? Comment évaluer la pertinence d'une règle ?


In [ ]:
def findContries(s): 
  #pattern2 = re.compile(r"([A-Z]\w+( [A-Z]\w+( [A-Z]\w+( [A-Z]\w+)?)?)?) is bordered")
  pattern = re.compile(r"{{flag\|([A-Z]\w+( [A-Z]\w+( [A-Z]\w+( [A-Z]\w+)?)?)?)}}")
  return set([str(x.group(0)).replace("{{flag|","").replace("}}","").replace(" is bordered","")for x in pattern.finditer(str(s))])

countries=set()
for page in list(df['text']):
  countries=countries.union(findContries(page))
print(len(countries))
countries
## patern 2 ( 155) ( intersection =1 avec la liste des countries )
#Pattern ( 728) ( intersection =43 avec la liste des countries )

728


{'ARG',
 'ASA',
 'ASEAN',
 'ASM',
 'AUS',
 'AUT',
 'AZE',
 'Aargau',
 'Abkhazia',
 'Abu Dhabi',
 'Aceh',
 'Acre',
 'Adygea',
 'Afghanistan',
 'African Union',
 'Aiti',
 'Akita',
 'Alabama',
 'Alagoas',
 'Alaska',
 'Albania',
 'Alberta',
 'Algeria',
 'Altai Republic',
 'Amapá',
 'Amazonas',
 'American Samoa',
 'Andalusia',
 'Andorra',
 'Angaur',
 'Angola',
 'Anguilla',
 'Antarctica',
 'Antwerp',
 'Aomori',
 'Aosta Valley',
 'Appenzell Ausserrhoden',
 'Appenzell Innerrhoden',
 'Apulia',
 'Arab League',
 'Aragon',
 'Argentina',
 'Arizona',
 'Arkansas',
 'Armenia',
 'Armenian SSR',
 'Artsakh',
 'Aruba',
 'Ascension Island',
 'Assyria',
 'Astrakhan Oblast',
 'Australasia',
 'Australia',
 'Australian Capital Territory',
 'Austria',
 'Austrian Empire',
 'Azad Kashmir',
 'Azerbaijan',
 'Azerbaijan SSR',
 'BEL',
 'BRA',
 'BUL',
 'Bahamas',
 'Bahia',
 'Bahrain',
 'Baker Island',
 'Balearic Islands',
 'Bangladesh',
 'Banten',
 'Barbados',
 'Bashkortostan',
 'Basque Country',
 'Batavian Republic',

In [ ]:
len(my_countries.intersection(countries))

43

2) Définir et implémenter des règles pour identifier des lieux, des organisations, des personnes (morts
et vivants), etc.


In [ ]:
def findLieu(s): 
  pattern2 = re.compile(r"in ([A-Z]\w+( [A-Z]\w+( [A-Z]\w+( [A-Z]\w+)?)?)?)")
  return set([str(x.group(0)).replace("in ","")for x in pattern2.finditer(str(s))])
lieu=set()
for page in list(df['text']):
  lieu=lieu.union(findLieu(page))
print(len(lieu))
lieu

42928


{'Kathamandu',
 'Champaign',
 'MCA',
 'Bye',
 'Baltimore County',
 'Grollman',
 'Weaver',
 'Saddlers',
 'Haaretz',
 'Social Science',
 'Rolex',
 'Epsom',
 'Karabakh',
 'Paterson',
 'Stroud',
 'Jones',
 'Schamehorn',
 'Matching',
 'Hartington',
 'DOD',
 'Natin',
 'Arctic',
 'Gerontology',
 'Wendell Welker',
 'Hadasha',
 'Liberal Democracies',
 'Lala',
 'Park Awards',
 'Franconia',
 'Pörssi',
 'Rücktritt',
 'Socotra',
 'Leland Stottlemeyer',
 'Soup',
 'Meldal',
 'The Court Jester',
 'Kumegawa',
 'North Eastern India',
 'Janesville',
 'French Dalmatia',
 'Hindenburg',
 'Shawangunk',
 'Parish Courthouse',
 'Büron',
 'Baulmes',
 'Freelan',
 'Rodríguez Cordero',
 'Mass Spectrometry Award',
 'Desert Hawk',
 'Central Mexico',
 'Royal Canon',
 'Keramics',
 'Driggs',
 'Date District',
 'TOP',
 'Heterosexual Marriage',
 'Molecular Biology',
 'Boyton',
 'Southeast Brazil',
 'Munich Rural District',
 'Ghouri',
 'Ramanathapuram',
 'Fundraising',
 'Sultana',
 'Gem',
 'Armadillos',
 'Wakasa Province',

In [45]:
def findPersons(s): 
  pattern = re.compile(r"( [A-Z]\w+)+? [A-Z]\w+ was born") #il est encadré par une virgule ou un point
  pattern2 = re.compile(r"( [A-Z]\w+)+? [A-Z]\w+ (is|was) (a|an|the) (politician|president|scientist)")
  return set([str(x.group(0)).replace(",","").replace(".","").replace("was born","")for x in pattern.finditer(str(s))]+[str(x.group(0)).replace("is","").replace("was","").replace("a","").replace("an","").replace("politician","").replace("president","").replace("scientist","")for x in pattern2.finditer(str(s))])

Persons=set()
for page in list(df['text']):
  Persons=Persons.union(findPersons(page))
print(len(Persons))
Persons
# avec pattern = re.compile(r"([A-Z]\w+)?( [A-Z]\w+)?( [A-Z]\w+)? [A-Z]\w+ was born") on a 2995 
# avec pattern = re.compile(r"( [A-Z]\w+)+? [A-Z]\w+ was born") on a 379 ( + qualitatif)

388


{' Actor Bruce Namet ',
 ' After Bridget ',
 ' After Helen ',
 ' Alan Cheuse ',
 ' Alberto Cappellari ',
 ' Alexander Crowley ',
 ' Ali Ahmed ',
 ' Ali Jinnah ',
 ' Ali Mazari ',
 ' Ali Shah ',
 ' Alice Ivers ',
 ' All Star ',
 ' Allan Poe ',
 ' Allan Törni ',
 ' Almeida Pessanha ',
 ' American Hairless Terrier ',
 ' Anatolyevich Sokurov ',
 ' Andra Pradesh ',
 ' Angela Cartwright ',
 ' Angelina Emily Grimke ',
 ' Angelo Medici ',
 ' Anisul Hoque ',
 ' Ann Bayley ',
 ' Ann Holloway ',
 ' Ann Kennedy ',
 ' Ann Laub ',
 ' Ann Vincent ',
 ' Annette Bullock ',
 ' Anthony De Niro ',
 ' Arthur Blair ',
 ' Arthur Brabazon Ponsonby ',
 ' Avril Lange ',
 ' Ayya Vaikundar ',
 ' Aziz Mian ',
 ' Bailey Aldrich ',
 ' Bancroft Hermansader ',
 ' Baptist Allgaier ',
 ' Basen Murmu ',
 ' Battista Cibo ',
 ' Baylón Chacón ',
 ' Beecher Stowe ',
 ' Before Walala ',
 ' Ben Ali ',
 ' Benedict XVI ',
 ' Birchman Saldivar ',
 ' Blaine Cram ',
 ' Boardman Hawes ',
 ' Bob Irwin ',
 ' Bon Jovi ',
 ' Bonham Cart

In [ ]:
def findOrganisations(s): 
  pattern1 = re.compile(r"([A-Z]\w+( [A-Z]\w+( [A-Z]\w+)?)?)? [A-Z]\w+ (is|was) (an|a|the)( organization| association| hopital| college| international organization| agency )")
  return set([str(x.group(0)).replace("is","").replace("was","").replace("a","").replace("an","").replace("the","").replace("organization","").replace("association","").replace("hopital","").replace("college","").replace("international organization","").replace("agency","")for x in pattern1.finditer(str(s))])
Organisations=set()
for page in list(df['text']): 
  Organisations=Organisations.union(findOrganisations(page))
print(len(Organisations))
Organisations


20


{' Fartuun is the organization',
 ' Initiative is an organization',
 ' It is an agency ',
 ' It is an organization',
 ' Nonviolence is an organization',
 ' Pugwash is an organization',
 ' SBS is an organization',
 ' There is a college',
 ' There is an association',
 ' This is an association',
 'Armenian Cultural Society is an organization',
 'Lauryn Ashby is a college',
 'Murray Edwards is a college',
 'Refuge Network is an association',
 'Sunset Park is an organization',
 'The CIS is an organization',
 'The FA is the association',
 'The GCC is an organization',
 'The Wikimedia Foundation is the organization',
 'The YMCA is an organization'}

[texte du lien](https://)3) Définir au moins 3 règles dans lesquelles l'entité nommée n'est pas dans une extrémité de la chaîne
de caractères.


In [49]:
def findPersons(s): 
  pattern = re.compile(r"(\.|,| )?( [A-Z]\w+)+? [A-Z]\w+ was born") #il est encadré par une virgule ou un point
  pattern2 = re.compile(r"( [A-Z]\w+)+? [A-Z]\w+ (is|was) (a|an|the) (politician|president|scientist)")
  return set([str(x.group(0)).replace(",","").replace(".","").replace("was born","")for x in pattern.finditer(str(s))]+[str(x.group(0)).replace("is","").replace("was","").replace("a","").replace("an","").replace("politician","").replace("president","").replace("scientist","")for x in pattern2.finditer(str(s))])

Persons=set()
for page in list(df['text']):
  Persons=Persons.union(findPersons(page))
print(len(Persons))
Persons
# avec pattern = re.compile(r"([A-Z]\w+)?( [A-Z]\w+)?( [A-Z]\w+)? [A-Z]\w+ was born") on a 2995 


388


{'  Anatolyevich Sokurov ',
 '  Her Highness The Queen Of North Last Heir Of The Great Stark Lineage Mellissa Elkin The Indestructible ',
 '  When Macbeth ',
 ' Actor Bruce Namet ',
 ' After Bridget ',
 ' After Helen ',
 ' Alan Cheuse ',
 ' Alberto Cappellari ',
 ' Alexander Crowley ',
 ' Ali Ahmed ',
 ' Ali Jinnah ',
 ' Ali Mazari ',
 ' Ali Shah ',
 ' Alice Ivers ',
 ' All Star ',
 ' Allan Poe ',
 ' Allan Törni ',
 ' Almeida Pessanha ',
 ' American Hairless Terrier ',
 ' Andra Pradesh ',
 ' Angela Cartwright ',
 ' Angelina Emily Grimke ',
 ' Angelo Medici ',
 ' Anisul Hoque ',
 ' Ann Bayley ',
 ' Ann Holloway ',
 ' Ann Kennedy ',
 ' Ann Laub ',
 ' Ann Vincent ',
 ' Annette Bullock ',
 ' Anthony De Niro ',
 ' Arthur Blair ',
 ' Arthur Brabazon Ponsonby ',
 ' Avril Lange ',
 ' Ayya Vaikundar ',
 ' Aziz Mian ',
 ' Bailey Aldrich ',
 ' Bancroft Hermansader ',
 ' Baptist Allgaier ',
 ' Basen Murmu ',
 ' Battista Cibo ',
 ' Baylón Chacón ',
 ' Beecher Stowe ',
 ' Before Walala ',
 ' Ben Ali

In [ ]:
def findOrganisations(s): 
  pattern1 = re.compile(r"(the|a|an| )?( [A-Z]\w+( [A-Z]\w+( [A-Z]\w+)?)?)? [A-Z]\w+ (is|was) (an|a|the)( international organization| organization| association| hopital| college| agency)")
  return set([str(x.group(0)).replace("is","").replace("was","").replace("an","").replace("a","").replace("the","").replace("organization","").replace("association","").replace("hopital","").replace("college","").replace("international organization","").replace("agency","")for x in pattern1.finditer(str(s))])
Organisations=set()
for page in list(df['text']): 
  Organisations=Organisations.union(findOrganisations(page))
print(len(Organisations))
Organisations


29


{'  The FA   ssocition',
 ' Ashby   ',
 ' Edwrds   ',
 ' FBI  n orgniztion',
 ' Frtuun   orgniztion',
 ' He   ',
 ' Inititive  n orgniztion',
 ' It  n gency',
 ' It  n interntionl orgniztion',
 ' It  n orgniztion',
 ' It  n ssocition',
 ' Joe   ',
 ' Nonviolence  n orgniztion',
 ' Olsen   ',
 ' Pugh  n orgniztion',
 ' Refuge Network  n ssocition',
 ' SBS  n orgniztion',
 ' SS   orgniztion',
 ' Sunset Prk  n orgniztion',
 ' Th   ',
 ' Th  n orgniztion',
 ' Th  n ssocition',
 ' The CIS  n orgniztion',
 ' The GCC  n orgniztion',
 ' The Wikimedi Foundtion   orgniztion',
 ' The YMCA  n orgniztion',
 ' There   ',
 ' There  n ssocition',
 'n Culturl Society  n orgniztion'}

In [ ]:
def findLieu(s): 
  pattern2 = re.compile(r"from ([A-Z]\w+( [A-Z]\w+( [A-Z]\w+( [A-Z]\w+)?)?)?)/ to")
  return set([str(x.group(0)).replace("from","").replace("to","")for x in pattern2.finditer(str(s))])

lieu=set()
for page in list(df['text']):
  lieu=lieu.union(findLieu(page))
print(len(lieu))
lieu

590022


{'[[Tyssestrengene Falls]]',
 '[[Latacunga]]',
 '[[Tarvagatai River]]',
 '[[Wiehenvenator]]',
 '[[Valcour Island]]',
 '[[Yumi Tsukiro]]',
 '[[Joseph Rabski]]',
 '[[George Brettgham Sowerby I]]',
 '[[meadow]]',
 '[[Trent Sasbury]]',
 '[[Henry Gordon Wells]]',
 '[[Marie Bashkirtseff]]',
 '[[Khan Sahib]]',
 '[[Wden am See]]',
 '[[Lucy Hood]]',
 '[[Georgian Football Federation]]',
 '[[Sven Epey]]',
 '[[Nothg  the Dark]]',
 '[[Capta Planet and the Planeteers]]',
 '[[Frank Dundr]]',
 '[[Tuti Sutiawati]]',
 '[[Amazon River]]',
 '[[Paktia Provce]]',
 '[[Billy Gunn]]',
 '[[chocolate cake]]',
 '[[Israel Academy of Sciences and Humanities]]',
 '[[Her Naked Sk]]',
 '[[warranty]]',
 '[[Committee for Investigation of National Aviation Accidents]]',
 '[[Igor Akfeev]]',
 '[[Seed bank]]',
 '[[unconstitutional]]',
 '[[Lance Percival]]',
 '[[Bluesville Records]]',
 '[[Pirgsdorf]]',
 '[[Vareia]]',
 '[[Fusa y the Place]]',
 '[[Boffa]]',
 '[[NGC 7090]]',
 '[[South West Hertfordshire]]',
 '[[Robert Ten Broec

4) Définir au moins 3 règles qu'utilise la ponctuation pour trouver les entités nommées.




In [ ]:
def findLieu(s): 
  pattern2 = re.compile(r"in ([A-Z]\w+( [A-Z]\w+( [A-Z]\w+( [A-Z]\w+)?)?)?)")
  return set([str(x.group(0)).replace("in","")for x in pattern.finditer(str(s))])

lieu=set()
for page in list(df['text']):
  lieu=lieu.union(findLieu(page))
print(len(lieu))
lieu

590880


{'[[Tyssestrengene Falls]]',
 '[[Latacunga]]',
 '[[Tarvagatai River]]',
 '[[Wiehenvenator]]',
 '[[Valcour Island]]',
 '[[meadow]]',
 '[[Henry Gordon Wells]]',
 '[[Marie Bashkirtseff]]',
 '[[Masolino]]',
 '[[Khan Sahib]]',
 '[[porcupine fish]]',
 '[[Lucy Hood]]',
 '[[Georgian Football Federation]]',
 '[[Frank Dundr]]',
 '[[Tuti Sutiawati]]',
 '[[Amazon River]]',
 '[[Billy Gunn]]',
 '[[chocolate cake]]',
 '[[Israel Academy of Sciences and Humanities]]',
 '[[warranty]]',
 '[[Committee for Investigation of National Aviation Accidents]]',
 '[[Seed bank]]',
 '[[unconstitutional]]',
 '[[Lance Percival]]',
 '[[Bluesville Records]]',
 '[[Walter Schmidinger]]',
 '[[Vareia]]',
 '[[Fusa y the Place]]',
 '[[Boffa]]',
 '[[NGC 7090]]',
 '[[South West Hertfordshire]]',
 '[[Robert Ten Broeck Stevens]]',
 '[[American Kickboxing Academy]]',
 '[[Parbold]]',
 '[[Clarence Brown]]',
 '[[Transpennine Express]]',
 '[[Lamplughsaura]]',
 '[[Sounding the Seventh Trumpet]]',
 '[[Davron]]',
 '[[Millpool]]',
 '[[Val

In [ ]:
def findPersons(s): 
  pattern = re.compile(r"name = [A-Z]\w+( [A-Z]\w+)+?") 
  return set([str(x.group(0)).replace("name = ","")for x in pattern.finditer(str(s))])
Persons=set()
for page in list(df['text']):
  Persons=Persons.union(findPersons(page))
print(len(Persons))
Persons
# avec pattern = re.compile(r"([A-Z]\w+)?( [A-Z]\w+)?( [A-Z]\w+)? [A-Z]\w+ was born") on a 2995 
# avec pattern = re.compile(r"( [A-Z]\w+)+? [A-Z]\w+ was born") on a 379 ( + qualitatif)

10702


{'Cesar Chavez',
 'Frederick McDonald',
 'Lana Therese',
 'Ben Stiller',
 'Al Jieulia',
 'Canes Venatici',
 'Alejandro Eduardo',
 'Bernie Kopell',
 'Janet Trafton',
 'Gloria Henry',
 'Sonny Black',
 'Marty Jackson',
 'Fatoş Bayer',
 'UK Garage',
 'Terry Wogan',
 'Susana Giménez',
 'Al Hunter',
 'Sergio Zarzar',
 'South Yemen',
 'Danny Glover',
 'Bluewater Highway',
 'International Astronomical',
 'Ivan Kostov',
 'Ibrahim Roshdy',
 'Royal Wootton',
 'John Holland',
 'George Burba',
 'Lawrence Pressman',
 'American Bandstand',
 'Only Human',
 'Hanne Karin',
 'Chinu Chandulal',
 'Saint Pierre',
 'The Craig',
 'Mount Kyllini',
 'Anquan Boldin',
 'Dimitrios Panousis',
 'Vassily Ivanchuk',
 'Albert Zafy',
 'Tarrant County',
 'Jimmy MacDonald',
 'Ted Christopher',
 'Hideto Matsumoto',
 'Francis Joseph',
 'Michio Kaku',
 'Boota Singh',
 'Kyle Russell',
 'Paul DelVecchio',
 'Femme Fatale',
 'Dave Camp',
 'Rubén José',
 'Andriy Danylko',
 'Alexei Petrovich',
 'Max Wright',
 'Francis Bacon',
 'Ja

In [50]:
def findOrganisations(s): 
  pattern1 = re.compile(r"(the|a|an| )?( [A-Z]\w+( [A-Z]\w+( [A-Z]\w+)?)?)? [A-Z]\w+ (is|was) (an|a|the)( international organization| organization| association| hopital| college)")
  return set([str(x.group(0))for x in pattern1.finditer(str(s))])
Organisations=set()
for page in list(df['text']): 
  Organisations=Organisations.union(findOrganisations(page))
print(len(Organisations))
Organisations


28


{'  The FA is the association',
 ' Ashby is a college',
 ' Edwards is a college',
 ' Fartuun is the organization',
 ' He was a college',
 ' Initiative is an organization',
 ' It is an organization',
 ' It was an association',
 ' It was an international organization',
 ' Joe was a college',
 ' Nonviolence is an organization',
 ' Olsen was a college',
 ' Pugwash is an organization',
 ' Refuge Network is an association',
 ' SBS is an organization',
 ' Sunset Park is an organization',
 ' The CIS is an organization',
 ' The GCC is an organization',
 ' The Wikimedia Foundation is the organization',
 ' The YMCA is an organization',
 ' There is a college',
 ' There is an association',
 ' This is an association',
 ' This was a college',
 ' This was an organization',
 'an Cultural Society is an organization',
 'the FBI was an organization',
 'the SS was the organization'}

5) Choisissez aléatoirement 10 pages Wikipédia et appliquez toutes les règles qui vous avez défini.
Imprimez les documents avec les entités nommées identifiées. Il y a des manquants ? Il y a des entités
nommées de trop ? Évaluez la performance des vos règles en calculant la [precision](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_score.html#sklearn.metrics.precision_score)  et le [rappel](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_score.html#sklearn.metrics.precision_score) de
votre implémentation pour ces 10 documents

In [68]:
#Calcul du rappel d'une règle
def rappel(truePos,falseNeg):
  return("le rappel est: ",truePos/(truePos+falseNeg))
#Calcul de la précision d'une règle
def precision(truePos,falsePos):
  return("la précision est: ",truePos/(truePos+falsePos))

In [13]:
import random

In [81]:
titles=list(df["title"])
random.shuffle(titles)
titles[:10]
pages2=df[df['title']==titre]['text'].values[0]
## ou
pages = df.sample(n=10)


In [82]:
import re
def findOrganisations(s): 
  pattern1 = re.compile(r"(the|a|an| )?( [A-Z]\w+( [A-Z]\w+( [A-Z]\w+)?)?)? [A-Z]\w+ (is|was) (an|a|the)( international organization| organization| association| hopital| college| agency)")
  return set([str(x.group(0)).replace("is","").replace("was","").replace("a","").replace("an","").replace("the","").replace("organization","").replace("association","").replace("hopital","").replace("college","").replace("international organization","").replace("agency","")for x in pattern1.finditer(str(s))])

def findLieu(s): 
  pattern2 = re.compile(r"in ([A-Z]\w+( [A-Z]\w+( [A-Z]\w+( [A-Z]\w+)?)?)?)/")
  return set([str(x.group(0)).replace("in","")for x in pattern2.finditer(str(s))])
  
def findPersons(s): 
  pattern = re.compile(r"name = [A-Z]\w+( [A-Z]\w+)+?") 
  return set([str(x.group(0)).replace("name = ","")for x in pattern.finditer(str(s))])

In [83]:
lieu=set()
Organisations=set()
persons=set()
persons=set()
for page in pages2:
  Organisations.union(findOrganisations(page))
  lieu=lieu.union(findLieu(page))
  persons=persons.union(findPersons(page))

print("persons=",persons2)
print("Organisations=",Organisations)
print("lieu=",lieu)

persons= {'Mexican Fritillary'}
Organisations= set()
lieu= set()


In [ ]:
#Il y a très peu d'entités nommées. Pour en avoir plus il faudrait que je recherche plus largement mes entités.

In [69]:
print("pour persons",rappel(1,0))
print("pour persons",precision(1,0))## on peut pas calcules les fales positive 

pour persons ('le rappel est: ', 1.0)
pour persons ('la précision est: ', 1.0)


In [ ]:
print("pour organisation",rappel(truePos,falseNeg))
print("pour lieu",rappel(truePos,falseNeg))